In [ ]:
from IPython.display import display, Javascript
import json

# This JavaScript runs automatically on notebook load
display(Javascript("""
(function() {
    // Check if we're in an iframe (JupyterLab compatibility)
    const inJupyterLab = window.self !== window.top;
    
    // Get session ID from URL
    const params = new URLSearchParams(window.location.search);
    const sessionId = params.get('session');
    
    // Function to handle code injection
    function injectAndRun(code) {
        // Create a new cell to avoid conflicts
        const newCell = Jupyter.notebook.insert_cell_below('code');
        newCell.set_text(code);
        
        // Execute after a short delay
        setTimeout(() => {
            newCell.execute();
            // Cleanup storage
            localStorage.removeItem(sessionId);
            localStorage.removeItem(sessionId + '_timestamp');
        }, 1000);
    }
    
    // Main execution flow
    if (sessionId) {
        const checkReady = () => {
            if (window.Jupyter && Jupyter.notebook) {
                const code = localStorage.getItem(sessionId);
                if (code) {
                    injectAndRun(code);
                } else {
                    console.warn('No code found for session:', sessionId);
                }
            } else {
                setTimeout(checkReady, 500);
            }
        };
        
        // Start checking for readiness
        setTimeout(checkReady, inJupyterLab ? 3000 : 1000);
    }
})();
"""))

# Default imports (always runs)
%matplotlib inline
import control
import matplotlib.pyplot as plt
print("Automatic execution in progress...")